In [1]:
import cobra
from cobra import io, flux_analysis
import os
#import pubchempy as pcp
#os.chdir(r"C:\Users\Bisbii\OneDrive - Universidade do Minho\Models")

In [123]:
model = cobra.io.read_sbml_model("yeast_lipidomics.xml")

In [124]:
def get_reaction_reactants_and_products(reaction):
    reactants = []
    products = []
    for metabolite in reaction.metabolites:
        if reaction.metabolites[metabolite] < 0:
            reactants.append(metabolite)
        else:
            products.append(metabolite)

    return reactants, products

In [125]:
sol = cobra.flux_analysis.pfba(model)
sol

,fluxes,reduced_costs
r_0001,0.000000,0.000000
r_0002,0.000000,-2.000000
r_0003,0.000000,-2.000000
r_0004,0.000000,-2.000000
r_0005,0.086385,-2.000000
...,...,...
r_4412,0.000000,52018.482587
r_4413,0.000000,89285.388998
r_4414,0.000000,114950.651367
r_4415,0.000000,178977.778852


In [126]:
slime_reactions = []
for reaction in model.reactions:
    if "SLIME" in reaction.name:
        slime_reactions.append(reaction.id)

In [127]:
for reaction in slime_reactions:
    if sol.to_frame().loc[reaction, "fluxes"] > 0:
        print(model.reactions.get_by_id(reaction).name)

1-phosphatidyl-1D-myo-inositol (1-16:0, 2-16:1) [cytoplasm] SLIME rxn
1-phosphatidyl-1D-myo-inositol (1-16:1, 2-16:1) [cytoplasm] SLIME rxn
ergosteryl oleate [endoplasmic reticulum membrane] SLIME rxn
stearate [cytoplasm] SLIME rxn
oleate [cytoplasm] SLIME rxn
phosphatidyl-L-serine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
phosphatidylcholine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
phosphatidylcholine (1-16:0, 2-18:1) [endoplasmic reticulum membrane] SLIME rxn
phosphatidylethanolamine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
triglyceride (1-16:0, 2-18:1, 3-16:0) [endoplasmic reticulum membrane] SLIME rxn
triglyceride (1-18:0, 2-18:1, 3-18:0) [endoplasmic reticulum membrane] SLIME rxn
triglyceride (1-18:0, 2-16:1, 3-18:1) [endoplasmic reticulum membrane] SLIME rxn
phosphatidylcholine (1-16:0, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn


In [106]:
sol["r_4228"]

27.661973440776222

In [128]:
sol_fba = model.optimize()

In [129]:
for reaction in slime_reactions:
    if sol_fba[reaction] > 0:
        print(model.reactions.get_by_id(reaction).name)
        print(sol_fba[reaction])

ergosteryl oleate [endoplasmic reticulum membrane] SLIME rxn
0.0005051341918858265
stearate [cytoplasm] SLIME rxn
0.00025377881921064303
oleate [cytoplasm] SLIME rxn
2.436734135231305e-06
phosphatidyl-L-serine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.00039892644210795574
phosphatidylcholine (1-16:0, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.0005301531492074693
phosphatidylcholine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.0011278645892237589
phosphatidylcholine (1-16:0, 2-18:1) [endoplasmic reticulum membrane] SLIME rxn
7.060558964749148e-05
phosphatidylethanolamine (1-16:1, 2-16:1) [endoplasmic reticulum membrane] SLIME rxn
0.0004942399758622675
triglyceride (1-16:0, 2-18:1, 3-16:0) [endoplasmic reticulum membrane] SLIME rxn
0.0004037385539625024
1-phosphatidyl-1D-myo-inositol (1-16:1, 2-16:1) [cytoplasm] SLIME rxn
0.0004201596756531305


In [143]:
fva = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=1, reaction_list=slime_reactions + ["r_2111"])

In [144]:
backbones_abundances = {}

for reaction in slime_reactions:
    if fva.loc[reaction, "maximum"] > 0:

        reaction = model.reactions.get_by_id(reaction)
        reactants, products = get_reaction_reactants_and_products(reaction)
        chains = {}
        backbone = ""
        for product in products:
            if "chain" in product.name:
                stoichiometry = reaction.get_coefficient(product)
                minimum = stoichiometry * fva.loc[reaction.id, "minimum"] / fva.loc["r_2111", "minimum"]
                maximum = stoichiometry * fva.loc[reaction.id, "maximum"] / fva.loc["r_2111", "maximum"]
                if product.name not in chains:
                    chains[product.name] = {"minimum": minimum, "maximum": maximum}
                else:
                    chains[product.name]["minimum"] += minimum
                    chains[product.name]["maximum"] += maximum
            else:
                backbone = product.name

        if backbone not in backbones_abundances:
            backbones_abundances[backbone] = chains
        else:
            for chain in chains:
                if chain not in backbones_abundances[backbone]:
                    backbones_abundances[backbone][chain] = chains[chain]
                else:
                    backbones_abundances[backbone][chain]["minimum"] += chains[chain]["minimum"]
                    backbones_abundances[backbone][chain]["maximum"] += chains[chain]["maximum"]


In [145]:
backbones_abundances

{'1-phosphatidyl-1D-myo-inositol [cytoplasm]': {'C16:1 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 2.293141112272747}},
 'ergosterol ester [cytoplasm]': {'C18:1 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 3.0609139339303466}},
 'fatty acid [cytoplasm]': {'C18:0 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 1.5487745525606211},
  'C18:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 0.01476564686119218}},
 'phosphatidyl-L-serine [cytoplasm]': {'C16:1 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 2.177254691917783}},
 'phosphatidylcholine [cytoplasm]': {'C16:0 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 3.3047925919009544},
  'C16:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 9.049104073803514},
  'C18:1 chain [cytoplasm]': {'minimum': 0.0, 'maximum': 0.4278420192515827}},
 'phosphatidylethanolamine [cytoplasm]': {'C16:1 chain [cytoplasm]': {'minimum': 0.0,
   'maximum': 2.6974554524221195}},
 'triglyceride [cytoplasm]': {'C16:0 chain [cytoplasm]': {'m

In [10]:
def getPatwhaysMap(model):
    res = {}
    for pathway in model.groups:
        for reaction in pathway.members:
            if reaction.id not in res.keys():
                res[reaction.id] = pathway.name
            else:
                res[reaction.id] += "; " + pathway.name
    return res

def save_simulation(model, sol, filename, caption):
    solution = pd.DataFrame(sol.fluxes.copy())
    solution["Reaction Name"] = [None for el in solution.index]
    reaction_list = solution.index
    for reaction in reaction_list:
        reaction_name = model.reactions.get_by_id(reaction).name
        solution["Reaction Name"].loc[solution.index == reaction] = reaction_name
    solution = solution.loc[round(solution["fluxes"],3)!=0]
    solution = solution.sort_values(by="fluxes", axis=0)
    map_path = getPatwhaysMap(model)
    df = pd.DataFrame.from_dict(data = map_path, orient = 'index', columns = ["Pathways"])
    final = solution.merge(df, left_index=True, right_index=True, how="left")
    final.to_excel(filename + ".xlsx")
    df = model.summary(sol).to_frame()
    df = df.loc[df["flux"] != 0]
    df = df.drop(["factor"], axis=1)
    df = df.sort_values(by="flux", axis=0, ascending=False)
    df.columns = ["Reaction", "Metabolite", "Flux"]
    metabolites_list = df["Metabolite"].tolist()
    for met in metabolites_list:
        df["Metabolite"].loc[df["Metabolite"] == met] = model.metabolites.get_by_id(df["Metabolite"].loc[df["Metabolite"] == met].values[0]).name
    df_styled = df.style.hide_index().set_caption(caption).background_gradient()
    dfi.export(df_styled, filename + ".tiff")

In [13]:
save_simulation(model, sol, "test", "a caption")

<IPython.core.display.Javascript object>

c:\users\bisbii\onedrive - universidade do minho\python scripts\python36\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


<IPython.core.display.Javascript object>

c:\users\bisbii\onedrive - universidade do minho\python scripts\python36\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [16]:
fva_pfba = fva.merge(sol.to_frame(), left_index=True, right_index=True)
fva_pfba.to_csv("fva_solution.tsv", sep="\t")
fva_pfba